<div style='background-image: url("./title01.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 200px'>
<div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
<div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computers, Waves, Simulations</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Finite-Difference Method - Acoustic Waves 2D - Heterogeneous case</div>
        </div>
    </div>
</div>

This exercise covers the following aspects:

* Presenting you with an implementation of the 2D acoustic wave equation 
* Allowing you to explore the benefits of using high-order finite-difference operators
* Understanding the concepts of stability (Courant criterion)
* Exploration of numerical dispersion and numerical grid anisotropy
* Changing the earth model and exploring some effects of structural heterogeneities (e.g., fault zones)

---

## Basic Equations

The acoustic wave equation in 2D is 
$$
\ddot{p}(x,z,t) \ = \ c(x,z)^2 (\partial_x^2 p(x,z,t) + \partial_z^2 p(x,z,t)) \ + s(x,z,t)
$$

and we replace the time-dependent (upper index time, lower indices space) part by

$$
 \frac{p_{j,k}^{n+1} - 2 p_{j,k}^n + p_{j,k}^{n-1}}{\mathrm{d}t^2} \ = \ c_j^2 ( \partial_x^2 p + \partial_z^2 p) \ + s_{j,k}^n
$$

solving for $p_{j,k}^{n+1}$. 
The extrapolation scheme is
$$
p_{j,k}^{n+1} \ = \ c_j^2 \mathrm{d}t^2 \left[ \partial_x^2 p + \partial_z^2 p \right]
+ 2p_{j,k}^n - p_{j,k}^{n-1} + \mathrm{d}t^2 s_{j,k}^n
$$
The  space derivatives are determined by 

$$
\partial_x^2 p \ = \ \frac{p_{j+1,k}^{n} - 2 p_{j,k}^n + p_{j-1,k}^{n}}{\mathrm{d}x^2}
$$
$$
\partial_z^2 p \ = \ \frac{p_{j,k+1}^{n} - 2 p_{j,k}^n + p_{j,k-1}^{n}}{\mathrm{d}z^2} 
$$

---

### 1. Getting started
Relate the time extrapolation loop with the numerical algorithm we developed in the course. Understand the input parameters for the simulation and the plots that are generated. Modify source and receiver locations and observe the effects on the seismograms. 

### 2. Stability
The Courant criterion is defined as $eps = (v \cdot dt) / dx$ and provides the maximum possible, stable time step, with $v$ beeing the velocity, $dt$ the time step and $dx$ the spatial step. Determine numerically the stability limit of the code as accurately as possible by increasing the time step. Print the max value of the pressure field at each time step and observe the evolution of it in the case of stable and unstable simulations. 


### 3. Numerical anisotropy
Increase the frequency of the wavefield by varying $f_0$. Investigate the angular dependence of the wavefield. Why does the wavefield look anisotropic? Which direction is the most accurate and why? What happens if you set the source time function to a spike (zero everywhere except one element with value $1$). 

### 4. Heterogeneous models
Change the various pre-implemented heterogeneous models. Invent a heterogeneous model of your liking and implement it appropriately. 



---

In [ ]:
# Import Libraries
# ----------------------------------------------
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots

Below is the 2D acoustic simulation code:

In [ ]:
# Simple finite difference solver 
# Acoustic wave equation  p_tt = c^2 p_xx + src
# 2-D regular grid

nx = 200      # grid points in x - 500
nz = 200      # grid points in z - 500
nt = 1000     # number of time steps
dx = 10.0     # grid increment in x - 1
dt = 0.001    # Time step
c0 = 3000.0   # velocity (can be an array) - 580
isx = nx // 2 # source index x - 250
isz = nz // 2 # source index z - 250
ist = 100     # shifting of source time function
f0 = 50.0     # dominant frequency of source (Hz)
isnap = 10    # snapshot frequency
T = 1.0 / f0  # dominant period
nop = 5       # length of operator

# Model type, available are "homogeneous", "fault_zone",
# "surface_low_velocity_zone", "random", "topography",
# "slab"
model_type = "topography"

# Receiver locations
irx = np.array([60, 80, 100, 120, 140])
irz = np.array([5, 5, 5, 5, 5])
seis = np.zeros((len(irx), nt))

# Initialize pressure at different time steps and the second
# derivatives in each direction
p = np.zeros((nz, nx))
pold = np.zeros((nz, nx))
pnew = np.zeros((nz, nx))
pxx = np.zeros((nz, nx))
pzz = np.zeros((nz, nx))


In [ ]:
# Initialize velocity model (the fun bit!)
c = np.zeros((nz, nx))

if model_type == "homogeneous":
    c += c0
elif model_type == "fault_zone":
    c += c0
    c[:, nx // 2 - 5: nx // 2 + 5] *= 0.8    
elif model_type == "surface_low_velocity_zone":
    c += c0
    c[1:10,:] *= 0.8
elif model_type == "random":
    pert = 0.4
    r = 2.0 * (np.random.rand(nz, nx) - 0.5) * pert
    c += c0 * (1 + r)   
elif model_type == "topography":
    c += c0
    c[0 : 10, 10 : 50] = 0                         
    c[0 : 10, 105 : 115] = 0                       
    c[0 : 30, 145 : 170] = 0
    c[10 : 40, 20 : 40]  = 0
    c[0 : 15, 50 : 105] *= 0.8    
elif model_type == "slab":
    c += c0
    c[110 : 125, 0 : 125] = 1.4 * c0
    for i in range(110, 180):
        c[i , i-5 : i + 15 ] = 1.4 * c0
else:
    raise NotImplementedError
    
cmax = c.max()

In [ ]:
# Source time function Gaussian, nt + 1 as we loose the last one by diff
src = np.empty(nt + 1)
for it in range(nt):
    src[it] = np.exp(-1.0 / T ** 2 * ((it - ist) * dt) ** 2)
# Take the first derivative
src = np.diff(src) / dt
src[nt - 1] = 0

In [ ]:
v = max([np.abs(src.min()), np.abs(src.max())])

# Create a color scale
colorscale = [[0, 'blue'], [0.5, 'white'], [1, 'red']]

# Create a 2D heatmap
heatmap = go.Heatmap(z=pnew, colorscale=colorscale, zmin=-v, zmax=v)

# Create annotations for receivers
annotations = [go.layout.Annotation(
    x=x, y=z, text='+', showarrow=False, font=dict(color='black')) for x, z in zip(irx, irz)]
annotations.append(go.layout.Annotation(
    x=isx, y=isz, text='o', showarrow=False, font=dict(color='black')))

# Output Courant criterion
print("Courant Criterion eps :")
print(cmax*dt/dx)

In [ ]:
ir = np.arange(len(irx))

# Time extrapolation
frames = []
for it in range(nt):
    if nop == 3:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(1, nx - 1):
            pzz[:, i] = p[:, i + 1] - 2 * p[:, i] + p[:, i - 1]
        for j in range(1, nz - 1):
            pxx[j, :] = p[j - 1, :] - 2 * p[j, :] + p[j + 1, :]

    if nop == 5:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(2, nx - 2):
            pzz[:, i] = -1./12*p[:, i+2]+4./3*p[:, i+1] - \
                5./2*p[:, i]+4./3*p[:, i-1]-1./12*p[:, i-2]
        for j in range(2, nz - 2):
            pxx[j, :] = -1./12*p[j+2, :]+4./3*p[j+1, :] - \
                5./2*p[j, :]+4./3*p[j-1, :]-1./12*p[j-2, :]

    pxx /= dx ** 2
    pzz /= dx ** 2

    # Time extrapolation
    pnew = 2 * p - pold + dt ** 2 * c ** 2 * (pxx + pzz)
    # Add source term at isx, isz
    pnew[isz, isx] = pnew[isz, isx] + src[it]

    # Save the state of the plot in a frame every isnap-th iteration
    if it % isnap == 0:
        frames.append(go.Frame(data=[go.Heatmap(z=pnew, colorscale=colorscale, zmin=-v, zmax=v)],
                               layout=go.Layout(title_text="Max P: %.2f" % p.max())))

    pold, p = p, pnew

    # Save seismograms
    seis[ir, it] = p[irz[ir], irx[ir]]

# Create layout
layout = go.Layout(
    title='2D Acoustic Finite Difference',
    annotations=annotations,
    width=600,
    height=600,
    xaxis=dict(title='ix', showline=True, linewidth=2, linecolor='black', mirror=True),
    yaxis=dict(title='iz', autorange='reversed', showline=True, linewidth=2, linecolor='black', mirror=True),
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None, {"frame": {"duration": 0, "redraw": True},
                                   "fromcurrent": True, "transition": {"duration": 0}}],
                      ),
                 dict(label="Pause",
                      method="animate",
                      args=[[None], {"frame": {"duration": 0, "redraw": False},
                                     "mode": "immediate",
                                     "transition": {"duration": 0}}],
                      )
                 ]
    )])
# Create figure
fig = go.Figure(data=[heatmap], layout=layout, frames=frames)

fig.show()

The cell below allows you to plot source time function, seismic velocites, and the resulting seismograms in windows inside the notebook. Remember to rerun after you simulated again!

In [ ]:
# Create subplot figure
fig = make_subplots(rows=2, cols=2)

# Add source time function plot
time = np.arange(nt) * dt
fig.add_trace(go.Scatter(x=time, y=src, mode='lines', name='Source time function', showlegend=False), row=1, col=1)

# Add seismograms plot
ymax = seis.ravel().max()
for ir in range(len(seis)):
    fig.add_trace(go.Scatter(x=time, y=seis[ir, :] + ymax * ir, mode='lines', name=f'Seismogram {ir}', showlegend=False), row=2, col=1)

# Add velocity model plot
fig.add_trace(go.Heatmap(z=c, colorscale='Viridis', name='Velocity Model', colorbar=dict(x=1, y=0.2, thickness=15, len=0.5)), row=2, col=2)

# Update layout
fig.update_layout(height=800, width=800, title_text="Source time function and Seismograms")
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_xaxes(title_text="ix", row=2, col=2)
fig.update_yaxes(title_text="Source amplitude", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=2, col=1)
fig.update_yaxes(title_text="iz", autorange="reversed", title_standoff=5, row=2, col=2)

fig.show()